In [12]:
import plotly.graph_objects as go
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf


# **Data**

In [13]:
ticker = "BTC-USD"

data = yf.download(ticker, start="2024-01-01", end="2024-12-27")

data.columns = [col[0] if col[1] == '' else col[0] for col in data.columns]


columns_to_keep = ['Open', 'High', 'Low', 'Close', 'Volume']
data = data[columns_to_keep]

data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2024-01-01,42280.234375,44175.437500,42214.976562,44167.332031,18426978443
2024-01-02,44187.140625,45899.707031,44176.949219,44957.968750,39335274536
2024-01-03,44961.601562,45503.242188,40813.535156,42848.175781,46342323118
2024-01-04,42855.816406,44770.023438,42675.175781,44179.921875,30448091210
2024-01-05,44192.980469,44353.285156,42784.718750,44162.691406,32336029347
...,...,...,...,...,...
2024-12-22,97218.320312,97360.265625,94202.187500,95104.937500,43147981314
2024-12-23,95099.390625,96416.210938,92403.132812,94686.242188,65239002919
2024-12-24,94684.343750,99404.062500,93448.015625,98676.093750,47114953674


# **Strategies**

In [14]:
class MyStrategy(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(period=20)
    def next(self):
        if not self.position and self.data.close > self.sma:
            self.buy()
        elif self.position and self.data.close < self.sma:
            self.sell()

class BuyAndHold(bt.Strategy):
    def __init__(self):
        pass

    def nextstart(self):
        self.order = self.buy()

# **BackTrader**

In [15]:
strategies = [MyStrategy, BuyAndHold]

strategy_dict = {}

for strategy in strategies:
    
    cerebro = bt.Cerebro()

    cerebro.addstrategy(strategy)

    data_feed = bt.feeds.PandasData(dataname=data)

    cerebro.adddata(data_feed)

    cerebro.broker.setcash(100000)

    cerebro.broker.setcommission(commission=0.001)

    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="timereturn")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")

    results = cerebro.run()

    strategy_dict[strategy.__name__] = {
        'cerebro': cerebro,
        'results': results
    }

In [16]:
# rtot: The total return for the entire backtest period (29.66%)
# ravg: The average per-bar return (e.g., if these are daily bars, this is the average daily return)
# rnorm: The normalized (annualized) return (23.00% per year, if daily bars)
# rnorm100: The same normalized return as a percentage (23.00%)

strategy_dict['MyStrategy']['results'][0].analyzers.returns.get_analysis()

OrderedDict([('rtot', 0.2965712287337731),
             ('ravg', 0.0008215269494010335),
             ('rnorm', 0.23001306496301024),
             ('rnorm100', 23.001306496301023)])

In [17]:
# 1) The current drawdown length (in bars).
# 2) The current drawdown in percentage (i.e., ~6.11%).
# 3) The current drawdown in monetary terms (e.g., $8,753.39).
# 4) The length (in bars) of the maximum drawdown observed.
# 5) The maximum drawdown percentage (~20.83%).
# 6) The maximum drawdown in monetary terms (e.g., $27,025.29).

strategy_dict['MyStrategy']['results'][0].analyzers.drawdown.get_analysis()

AutoOrderedDict([('len', 9),
                 ('drawdown', 6.10940833578134),
                 ('moneydown', 8753.390945312509),
                 ('max',
                  AutoOrderedDict([('len', 242),
                                   ('drawdown', 20.830351094452965),
                                   ('moneydown', 27025.291929687504)]))])

In [18]:
strategy_dict['BuyAndHold']['results'][0].analyzers.drawdown.get_analysis()

AutoOrderedDict([('len', 9),
                 ('drawdown', 6.389433844585417),
                 ('moneydown', 10345.0859375),
                 ('max',
                  AutoOrderedDict([('len', 237),
                                   ('drawdown', 14.85015406565669),
                                   ('moneydown', 19134.74609375)]))])

In [19]:
strategy_dict['BuyAndHold']['results'][0].analyzers.timereturn.get_analysis()

OrderedDict([(datetime.datetime(2024, 1, 1, 0, 0), 0.0),
             (datetime.datetime(2024, 1, 2, 0, 0), 0.0072664098437500435),
             (datetime.datetime(2024, 1, 3, 0, 0), -0.0209457294329638),
             (datetime.datetime(2024, 1, 4, 0, 0), 0.01350424517257931),
             (datetime.datetime(2024, 1, 5, 0, 0), -0.00017239330781526618),
             (datetime.datetime(2024, 1, 6, 0, 0), -0.0017361525694168378),
             (datetime.datetime(2024, 1, 7, 0, 0), -0.0004620954455381021),
             (datetime.datetime(2024, 1, 8, 0, 0), 0.03036157369707504),
             (datetime.datetime(2024, 1, 9, 0, 0), -0.008086238071082774),
             (datetime.datetime(2024, 1, 10, 0, 0), 0.0047890739335965105),
             (datetime.datetime(2024, 1, 11, 0, 0), -0.0025312538405176666),
             (datetime.datetime(2024, 1, 12, 0, 0), -0.03441856607149474),
             (datetime.datetime(2024, 1, 13, 0, 0), -0.00010935870013972782),
             (datetime.datetime(2024, 1

In [20]:
# Create a Plotly figure
fig = go.Figure()

for strat_name, data_dict in strategy_dict.items():
    run_results = data_dict['results']
    
    # Usually returns is a list of strategy instances; we assume one strategy instance
    strategy_instance = run_results[0]
    
    # Extract the TimeReturn analyzer
    time_return_dict = strategy_instance.analyzers.timereturn.get_analysis()
    
    # Convert to a pandas Series
    returns_series = pd.Series(time_return_dict)
    
    # Calculate cumulative returns: (1 + r).cumprod() - 1
    cumulative_returns = (1 + returns_series).cumprod() - 1
    
    # Plot with Plotly
    fig.add_trace(go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns.values,
        mode='lines',
        name=f"{strat_name} (Cumulative)"
    ))

# Update figure layout
fig.update_layout(
    title="Strategy Comparison: Cumulative Returns",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    template="plotly_white",
    width=900,
    height=500
)

# Show the figure
fig.show()
